In [1]:
import os
import pandas as pd
import numpy as np
#import re
PUIdata = os.getenv('PUIDATA')

In [2]:
data = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9')

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,17,39,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#1. borough numbers
#community district
data['Borough'].drop_duplicates()

0          BROOKLYN
1       Unspecified
5     STATEN ISLAND
7         MANHATTAN
10            BRONX
13           QUEENS
Name: Borough, dtype: object

In [4]:
CB = data['Community Board']
print CB.drop_duplicates().shape
#CB1 = CB.drop('Unspecified')

(77,)


In [5]:
CB1 = CB.drop_duplicates()
CB1 = CB1.reset_index()

In [6]:
number = []
district = []
for i in range(0,len(CB1)):
    number.append(CB1.iloc[i,1].split(' ')[0])
    district.append(CB1.iloc[i,1].split(' ')[1])

In [7]:
CB2 = pd.DataFrame([number, district]).T
CB2 = CB2[CB2 != 'Unspecified']
CB2 = CB2.dropna()

In [8]:
CB2.dropna().shape

(71, 2)

In [9]:
#sort the data by occurance number
rank = data.groupby('Community Board').count()['Unique Key']

In [10]:
rank2 = rank.copy()
rank2.sort(ascending=False)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  from ipykernel import kernelapp as app


In [11]:
#complaints made by each community district rank:
print rank2

Community Board
0 Unspecified                1109446
12 MANHATTAN                  268053
Unspecified BROOKLYN          257844
12 QUEENS                     229383
Unspecified MANHATTAN         229164
Unspecified QUEENS            218268
Unspecified BRONX             199775
03 BROOKLYN                   197306
07 QUEENS                     195961
17 BROOKLYN                   191720
05 BROOKLYN                   189765
14 BROOKLYN                   187794
07 BRONX                      185729
01 BROOKLYN                   185057
01 STATEN ISLAND              182713
04 BRONX                      181953
01 QUEENS                     171484
05 QUEENS                     168570
12 BRONX                      168455
13 QUEENS                     163851
05 BRONX                      161078
18 BROOKLYN                   156100
07 MANHATTAN                  151419
09 QUEENS                     151242
12 BROOKLYN                   150320
03 MANHATTAN                  150296
08 MANHATTAN          

In [14]:
#load the other two dataframes

df_pop = pd.read_csv(PUIdata + "/Final_Demographics.csv")
df_infr = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [15]:
#add a community district column in df_pop
#keep the community district indicator consistent with original data format
cd_id = df_pop['cd_id']
bo = []
num = []
for i in range(0,len(cd_id)):
    bo.append(cd_id[i][:2])
    num.append(cd_id[i][2:])

In [16]:
bo = pd.DataFrame([num, bo]).T

bo = bo.replace('BX','BRONX')
bo = bo.replace('BK','BROOKLYN')
bo = bo.replace('MN', 'MANHATTAN')
bo = bo.replace('SI', 'STATEN ISLAND')
bo = bo.replace('QN', 'QUEENS')

In [17]:
bo['community district'] = [bo[0][i]+ ' '+ bo[1][i] for i in range(0,59)]
bo['pop density'] = df_pop['Population Density (per sq. mile)']
bo.drop([0,1], axis = 1,inplace = True)
bo.head()

,community district,pop density
0,08 BRONX,31229.95006
1,12 BRONX,19966.67839
2,10 BRONX,12913.81703
3,11 BRONX,35677.95453
4,03 BRONX,39405.79222


In [18]:
rank3 = pd.DataFrame(rank2)
rank3['community district'] = rank3.index

In [19]:
#merge 311 data with population data
merge1 = pd.merge(rank3, bo)
merge1.head()

,Unique Key,community district,pop density
0,268053,12 MANHATTAN,75217.43287
1,229383,12 QUEENS,27089.67720
2,197306,03 BROOKLYN,58004.86816
3,195961,07 QUEENS,21836.66849
4,191720,17 BROOKLYN,44659.44461


In [20]:
# select needed columns in df_infr1
df_infr1 = df_infr[['Qualifying Name', 'Households: With An Internet Subscription']]

In [21]:
# keep community district indicator consistent with 311 data
infr_loc = []
for i in range(0, len(df_infr1)):
    infr_loc.append(df_infr1['Qualifying Name'][i].split('-')[1].split(' ')[-1] + ' ' + 
                    df_infr1['Qualifying Name'][i].split('-')[1].split(' ')[0].upper())

# drop uncesssary column 
df_infr1['community district'] = infr_loc
df_infr1.drop('Qualifying Name', axis = 1, inplace = True)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
#data merge1 with df_infr1
merge2 = pd.merge(merge1, df_infr1)
merge2 = merge2.rename(columns = {'Unique Key': 'Complaint number'})
merge2.head()

,Complaint number,community district,pop density,Households: With An Internet Subscription
0,268053,12 MANHATTAN,75217.43287,56940
1,229383,12 QUEENS,27089.67720,53848
2,191720,17 BROOKLYN,44659.44461,35675
3,187794,14 BROOKLYN,62164.35980,45090
4,168455,12 BRONX,19966.67839,32243
